# DGP for regression

Here we'll show the DGP for regression, using small to medium data sets. 

In [1]:
import sys
sys.path.append('../src')

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline 

from GPflow.likelihoods import Gaussian
from GPflow.kernels import RBF, White
from GPflow.mean_functions import Constant
from GPflow.sgpr import SGPR, GPRFITC
from GPflow.svgp import SVGP
from GPflow.gpr import GPR

from scipy.cluster.vq import kmeans2

from get_data import get_regression_data
from dgp import DGP
import time


Let's use the kin8nm data set

In [2]:
X, Y, Xs, Ys = get_regression_data('kin8nm', split=0)
print 'N: {}, D: {}, Ns: {}'.format(X.shape[0], X.shape[1], Xs.shape[0])

N: 7372, D: 8, Ns: 820


## Single layer models

Our baseline model is a sparse GP, but since the dataset is small we can also train without minibatches so we'll also compare to a collapsed sparse GP (with analytically optimal $q(\mathbf u)$) which is known as SGPR in GPflow terminology, and we'll also cpmpare to FITC

In [3]:
Z_100 = kmeans2(X, 100, minit='points')[0]

D = X.shape[1]
Y_mean, Y_std = np.average(Y), np.std(Y) 

m_sgpr = SGPR(X, Y, RBF(D, variance=Y_std**2), Z_100.copy(), mean_function=Constant(Y_mean))
m_svgp = SVGP(X, Y, RBF(D, variance=Y_std**2), Gaussian(), Z_100.copy(), mean_function=Constant(Y_mean), q_diag=False, whiten=True)
m_fitc = GPRFITC(X, Y, RBF(D, variance=Y_std**2), Z_100.copy(), mean_function=Constant(Y_mean))

for m in [m_sgpr, m_svgp, m_fitc]:
    m.mean_function.fixed = True
    m.likelihood.variance = 0.1 * Y_std

## DGP models

We'll include a DGP with a single layer here for comparision. We've used a large minibatch size of $\text{min}(10000, N)$, but it works just fine for smaller batches

In [4]:
def make_dgp(L):
    kernels = []
    for l in range(L):
        kernels.append(RBF(D, lengthscales=1., variance=1.))
    mb = 10000 if X.shape[0] > 10000 else None
    model = DGP(X, Y, Z_100, kernels, Gaussian(), num_samples=1, minibatch_size=mb)

    model.layers[-1].kern.variance = Y_std**2
    model.likelihood.variance = Y_std*0.1 
    model.layers[-1].mean_function = Constant(Y_mean)
    model.layers[-1].mean_function.fixed = True
    
    # start the inner layers deterministically 
    for layer in model.layers[:-1]:
        layer.q_sqrt = layer.q_sqrt.value * 1e-5
    
    return model

m_dgp1 = make_dgp(1)
m_dgp2 = make_dgp(2)
m_dgp3 = make_dgp(3)
m_dgp4 = make_dgp(4)
m_dgp5 = make_dgp(5)

## Prediction

We'll calculate test rmse and likelihood in batches (so the larger datasets don't cause memory problems)

For the DGP models we need to take an average over the samples for the rmse. The `predict_density` function already does this internally


In [5]:
def batch_assess(model, assess_model, X, Y):
    n_batches = max(int(X.shape[0]/1000.), 1)
    lik, sq_diff = [], []
    for X_batch, Y_batch in zip(np.array_split(X, n_batches), np.array_split(Y, n_batches)):
        l, sq = assess_model(model, X_batch, Y_batch)
        lik.append(l)
        sq_diff.append(sq)
    lik = np.concatenate(lik, 0)
    sq_diff = np.array(np.concatenate(sq_diff, 0), dtype=float)
    return np.average(lik), np.average(sq_diff)**0.5

def assess_single_layer(model, X_batch, Y_batch):
    lik = m.predict_density(Xs, Ys)
    mean, var = m.predict_y(Xs)
    sq_diff = ((mean - Ys)**2)
    return lik, sq_diff 

S = 100
def assess_sampled(model, X_batch, Y_batch):
    lik = m.predict_density(Xs, Ys, S)
    mean_samples, var_samples = m.predict_y(Xs, 100)
    mean = np.average(mean_samples, 0)
    sq_diff = ((mean - Ys)**2)
    return lik, sq_diff 


## Training 

We'll optimize single layer models and using LFBGS and the dgp models with Adam. It will be interesting to compare the result of `m_svgp` compared to `m_dgp1`: if there is a difference it will be down to the optimizer. 

We need to take the data in batches to predict for the larger data sets, so we'll define that now

In [6]:
for m, name in zip([m_sgpr, m_svgp, m_fitc], ['col sgp', 'sgp', 'fitc']):
    t = time.time()
    m.optimize()
    lik, rmse = batch_assess(m, assess_single_layer, Xs, Ys)
    print '{}  lik: {:.4f}, rmse: {:.4f}. Training time: {:.4f}'.format(name, lik, rmse, time.time() - t)


col sgp  lik: 0.9770, rmse: 0.0865. Training time: 13.6648
sgp  lik: 0.9670, rmse: 0.0874. Training time: 14.7417
fitc  lik: 1.1348, rmse: 0.0826. Training time: 12.8889


Now for the DGP models:

In [7]:
for m, name in zip([m_dgp1, m_dgp2, m_dgp3, m_dgp4, m_dgp5], ['dgp1', 'dgp2', 'dgp3', 'dgp4', 'dgp5']):
    t = time.time()
    m.optimize(tf.train.AdamOptimizer(0.01), maxiter=5000)
    lik, rmse = batch_assess(m, assess_sampled, Xs, Ys)
    print '{}  lik: {:.4f}, rmse: {:.4f}. Training time: {:.4f}'.format(name, lik, rmse, time.time() - t)


dgp1  lik: 0.9721, rmse: 0.0870. Training time: 60.8755
dgp2  lik: 1.3025, rmse: 0.0659. Training time: 193.0561
dgp3  lik: 1.3336, rmse: 0.0638. Training time: 323.3256
dgp4  lik: 1.3309, rmse: 0.0642. Training time: 455.6249
dgp5  lik: 1.3307, rmse: 0.0641. Training time: 692.9579


We see an improvement in the 2 and 3 layers, but after that additional layers don't add anything 